In [1]:
# Prior to running the code, make sure to download sample_<year>.zip from the Freddie Mac website and unzip them and store them all in the same folder. 
# Provide below the name for the folder where sample_<year> folder are all stored. The output csv file will be copied in this folder. 
 
folder_path = '/Users/yuyaogawa/Documents/Home Work/portfolio/loan_forecast/fraddie_mae_data'

# Now specify the minimum year and maximum year of data you are working with. 

min_year = 1999
max_year = 2022

# At each vintage (quarterly), there are 12500 unique loans. If sample year becomes larger, it quickly becomes computationally infeasible. 
# For that reason, you can specify the number of loans you want to randomly select out of 12500 of the same vintage to work with a subsample.

sample_size = 1000    # If you would like to work with a full dataset, set sample_size = 12500.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# Function to read the file and process its contents
def parse_origination(filename, year):
    data = []
    with open(filename, 'r') as file:
        for line in file:
            # Remove spaces and split by '|'
            processed_line = ''.join(line.split()).split('|')
            if len(processed_line) == 32:
                data.append(processed_line)
    
    # Create a DataFrame with 32 columns. Variable definitions and how the data are stored in the txt file is available in Freddie Mac.  
    df = pd.DataFrame(data, columns=['CREDIT_SCORE', 'FIRST_PAYMENT_DATE', 'FIRST_TIME_HOMEBUYER_FLAG', 'MATURITY_DATE', 'METROPOLITAN_DIVISION',
                                     'MORTGAGE_INSURANCE_PERCENTAGE', 'NUMBER_OF_UNITS', 'OCCUPANCY_STATUS', 'ORIGINAL_COMBINED_LOAN-TO-VALUE',
                                     'ORIGINAL_DEBT-TO-INCOME', 'ORIGINAL_UPB', 'ORIGINAL_LOAN-TO-VALUE', 'ORIGINAL_INTEREST_RATE', 'CHANNEL',
                                     'PREPAYMENT_PENALTY_MORTGAGE', 'AMORTIZATION_TYPE', 'PROPERTY_STATE', 'PROPERTY_TYPE', 'POSTAL_CODE',
                                     'LOAN_SEQUENCE_NUMBER', 'LOAN_PURPOSE', 'ORIGINAL_LOAN_TERM', 'NUMBER_OF_BORROWERS', 'SELLER_NAME', 
                                     'SERVICER_NAME', 'SUPER_CONFORMING_FLAG', 'PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER', 'PROGRAM_INDICATOR',
                                     'RELIEF_REFINANCE_INDICATOR', 'PROPERTY_VALUATION METHOD', 'INTEREST_ONLY_INDICATOR', 'MI_CANCELLATION_INDICATOR'])
    # Assigns the quarter in which loan is originated to the year. 
    df['VINTAGE'] = str(year) + df['LOAN_SEQUENCE_NUMBER'].str[3:5]
    return df


In [6]:
# For loop to create a master dataframe that contains all of the origination data. 
origination_data = pd.DataFrame([])
for year in range(min_year, max_year+1):
    filename = f'{folder_path}/sample_{year}/sample_orig_{year}.txt'
    origination_data = pd.concat([origination_data, parse_origination(filename, year)], ignore_index=True)
origination_data.head()

,CREDIT_SCORE,FIRST_PAYMENT_DATE,FIRST_TIME_HOMEBUYER_FLAG,MATURITY_DATE,METROPOLITAN_DIVISION,MORTGAGE_INSURANCE_PERCENTAGE,NUMBER_OF_UNITS,OCCUPANCY_STATUS,ORIGINAL_COMBINED_LOAN-TO-VALUE,ORIGINAL_DEBT-TO-INCOME,...,SELLER_NAME,SERVICER_NAME,SUPER_CONFORMING_FLAG,PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER,PROGRAM_INDICATOR,RELIEF_REFINANCE_INDICATOR,PROPERTY_VALUATION METHOD,INTEREST_ONLY_INDICATOR,MI_CANCELLATION_INDICATOR,VINTAGE
0,618,200210,N,202902,10420,25,1,P,85,24,...,Othersellers,Otherservicers,,,9,,9,N,9,1999Q1
1,619,200303,N,201802,,000,1,P,74,25,...,Othersellers,Otherservicers,,,9,,9,N,9,1999Q1
2,515,200303,N,202903,12700,000,1,S,78,32,...,Othersellers,Otherservicers,,,9,,9,N,9,1999Q1
3,649,199904,Y,202903,17820,999,1,P,95,46,...,Othersellers,GMACMORTGAGECORPORATION,,,9,,9,N,9,1999Q1
4,768,199905,N,202904,12980,000,1,P,75,21,...,Othersellers,Otherservicers,,,9,,9,N,9,1999Q1


In [7]:
list = []
for year in range(min_year, max_year+1):
    for i in range(1,5):
        list.append(len(origination_data[origination_data['VINTAGE'] == f'{year}Q{i}']['LOAN_SEQUENCE_NUMBER'].unique()))
pd.DataFrame(list).describe()
# Every vintage has 12500 unique loans

,0
count,96.0
mean,12500.0
std,0.0
min,12500.0
25%,12500.0
50%,12500.0
75%,12500.0
max,12500.0


In [8]:
# At each vintage, randomly select the loan to construct a subsample with the size of 4000 to reduce the sample size (whose size is 12500 at each vintage.)
import random as rand

rand_origination = pd.DataFrame([])
for year in range(min_year, max_year+1):
    for i in range(1,5):
        list = [a for a in origination_data[origination_data['VINTAGE'] == f'{year}Q{i}']['LOAN_SEQUENCE_NUMBER']]
        randomlist = rand.sample(list, sample_size)
        data = pd.DataFrame({'LOAN_SEQUENCE_NUMBER':randomlist})
        data['VINTAGE'] = f'{year}Q{i}'
        rand_origination = pd.concat([rand_origination, data], ignore_index=True)

In [9]:
# Now that I have a smaller random sample, I can left join to the original data. 
rand_origination

,LOAN_SEQUENCE_NUMBER,VINTAGE
0,F99Q10470228,1999Q1
1,F99Q10580121,1999Q1
2,F99Q10434495,1999Q1
3,F99Q10505105,1999Q1
4,F99Q10199156,1999Q1
...,...,...
95995,F22Q40006425,2022Q4
95996,F22Q40026526,2022Q4
95997,F22Q40091690,2022Q4
95998,F22Q40092625,2022Q4


In [10]:
rand_origination = pd.merge(rand_origination, origination_data, on =['VINTAGE','LOAN_SEQUENCE_NUMBER'], how = 'inner')
rand_origination

,LOAN_SEQUENCE_NUMBER,VINTAGE,CREDIT_SCORE,FIRST_PAYMENT_DATE,FIRST_TIME_HOMEBUYER_FLAG,MATURITY_DATE,METROPOLITAN_DIVISION,MORTGAGE_INSURANCE_PERCENTAGE,NUMBER_OF_UNITS,OCCUPANCY_STATUS,...,NUMBER_OF_BORROWERS,SELLER_NAME,SERVICER_NAME,SUPER_CONFORMING_FLAG,PRE-RELIEF_REFINANCE_LOAN_SEQUENCE_NUMBER,PROGRAM_INDICATOR,RELIEF_REFINANCE_INDICATOR,PROPERTY_VALUATION METHOD,INTEREST_ONLY_INDICATOR,MI_CANCELLATION_INDICATOR
0,F99Q10470228,1999Q1,701,199905,N,201404,,000,1,P,...,02,FLEETMORTGAGECORPORATION,WASHINGTONMUTUALBANK,,,9,,9,N,9
1,F99Q10580121,1999Q1,758,199905,N,201904,,000,1,P,...,02,"LASALLEBANK,FSB","ABNAMROMORTGAGEGROUP,INC.",,,9,,9,N,9
2,F99Q10434495,1999Q1,741,199905,N,202904,,000,1,P,...,02,FIRSTUNIONMORTGAGECORPORATION,BRANCHBANKING&TRUSTCOMPANY,,,9,,9,N,9
3,F99Q10505105,1999Q1,706,199903,N,202902,33460,000,1,P,...,01,"LASALLEBANK,FSB",CHASEMANHATTANMORTGAGECORPORATION,,,9,,9,N,9
4,F99Q10199156,1999Q1,562,199905,N,201404,,000,1,P,...,02,Othersellers,Otherservicers,,,9,,9,N,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95995,F22Q40006425,2022Q4,687,202212,N,205211,,000,1,I,...,02,NATIONSTARMORTGAGELLCDBAMR.COOPER,NATIONSTARMORTGAGELLCDBAMR.COOPER,,,9,,2,N,7
95996,F22Q40026526,2022Q4,675,202212,N,205211,40140,000,1,P,...,02,Othersellers,Otherservicers,,,9,,2,N,7
95997,F22Q40091690,2022Q4,738,202301,N,205212,33340,30,1,P,...,02,NEWREZLLC,NEWRESIDENTIALMORTGAGELLC,,,9,,2,N,N
95998,F22Q40092625,2022Q4,784,202301,Y,205212,35154,30,1,P,...,01,Othersellers,"LAKEVIEWLOANSERVICING,LLC",,,9,,2,N,N


In [12]:
# Now that we have the origination data, we can parse the monthly performance data file

def parse_performance(filename):
    data = []
    with open(filename, 'r') as file:
        for line in file:
            # Remove spaces and split by '|'
            processed_line = ''.join(line.split()).split('|')
            if len(processed_line) == 32:
                data.append(processed_line)
    
    # Create a DataFrame with 5 columns
    df = pd.DataFrame(data,columns=['LOAN_SEQUENCE_NUMBER', 'MONTHLY_REPORTING_PERIOD', 'CURRENT_ACTUAL_UPB', 'CURRENT_LOAN_DELINQUENCY_STATUS', 'LOAN_AGE',
                                    'REMAINING_MONTHS_TO_LEGAL_MATURITY', 'DEFECT_SETTLEMENT_DATE', 'MODIFICATION_FLAG', 'ZERO_BALANCE_CODE',
                                    'ZERO_BALANCE_EFFECTIVE_DATE', 'CURRENT_INTEREST_RATE', 'CURRENT_NON-INTEREST_BEARING_UPB', 'DUE_DATE_OF_LAST_PAID_INSTALLMENT', 
                                    'MI_RECOVERIES', 'NET_SALE_PROCEEDS', 'NON_MI_RECOVERIES', 'TOTAL_EXPENSES', 'LEGAL_COSTS', 'MAINTENANCE_AND_PRESERVATION_COSTS',
                                    'TAXES_AND_INSURANCE', 'MISCELLANEOUS_EXPENSES', 'ACTUAL_LOSS_CALCULATION', 'CUMULATIVE_MODIFICATION_COST', 'STEP_MODIFICATION_FLAG', 
                                    'PAYMENT_DEFERRAL', 'ESTIMATED_LOAN_TO_VALUE', 'ZERO_BALANCE_REMOVAL', 'DELINQUENT_ACCRUED_INTEREST',
                                    'DELINQUENCY_DUE_TO_DISASTER', 'BORROWER_ASSISTANCE_STATUS_CODE', 'CURRENT_MONTH_MODIFICATION_COST', 'INTEREST_BEARING_UPB']
                                    )
    df['VINTAGE'] = str(year) + df['LOAN_SEQUENCE_NUMBER'].str[3:5]
    return df

In [13]:
# Replace with your file path
performance_data = pd.DataFrame([])
for year in range(min_year, max_year+1):
    filename = f'{folder_path}/sample_{year}/sample_svcg_{year}.txt'
    a = pd.merge(rand_origination, parse_performance(filename), on =['LOAN_SEQUENCE_NUMBER'], how = 'inner')
    performance_data = pd.concat([performance_data, a], ignore_index=True)
    print(f'Parsing the loan data in year {year} is done.')

print('Finished.')
performance_data.head()

Parsing the loan data in year 1999 is done.
Parsing the loan data in year 2000 is done.
Parsing the loan data in year 2001 is done.
Parsing the loan data in year 2002 is done.
Parsing the loan data in year 2003 is done.
Parsing the loan data in year 2004 is done.
Parsing the loan data in year 2005 is done.
Parsing the loan data in year 2006 is done.
Parsing the loan data in year 2007 is done.
Parsing the loan data in year 2008 is done.
Parsing the loan data in year 2009 is done.
Parsing the loan data in year 2010 is done.
Parsing the loan data in year 2011 is done.
Parsing the loan data in year 2012 is done.
Parsing the loan data in year 2013 is done.
Parsing the loan data in year 2014 is done.
Parsing the loan data in year 2015 is done.
Parsing the loan data in year 2016 is done.
Parsing the loan data in year 2017 is done.
Parsing the loan data in year 2018 is done.
Parsing the loan data in year 2019 is done.
Parsing the loan data in year 2020 is done.
Parsing the loan data in year 20

,LOAN_SEQUENCE_NUMBER,VINTAGE_x,CREDIT_SCORE,FIRST_PAYMENT_DATE,FIRST_TIME_HOMEBUYER_FLAG,MATURITY_DATE,METROPOLITAN_DIVISION,MORTGAGE_INSURANCE_PERCENTAGE,NUMBER_OF_UNITS,OCCUPANCY_STATUS,...,STEP_MODIFICATION_FLAG,PAYMENT_DEFERRAL,ESTIMATED_LOAN_TO_VALUE,ZERO_BALANCE_REMOVAL,DELINQUENT_ACCRUED_INTEREST,DELINQUENCY_DUE_TO_DISASTER,BORROWER_ASSISTANCE_STATUS_CODE,CURRENT_MONTH_MODIFICATION_COST,INTEREST_BEARING_UPB,VINTAGE_y
0,F99Q10470228,1999Q1,701,199905,N,201404,,000,1,P,...,,,,,,,,,124000.00,1999Q1
1,F99Q10470228,1999Q1,701,199905,N,201404,,000,1,P,...,,,,,,,,,123000.00,1999Q1
2,F99Q10470228,1999Q1,701,199905,N,201404,,000,1,P,...,,,,,,,,,123000.00,1999Q1
3,F99Q10470228,1999Q1,701,199905,N,201404,,000,1,P,...,,,,,,,,,122000.00,1999Q1
4,F99Q10470228,1999Q1,701,199905,N,201404,,000,1,P,...,,,,,,,,,122000.00,1999Q1


In [ ]:
# For convinience, save the data in csv so that we can work with it without changing the original data. 
performance_data.to_csv('loan_performance.csv', index = False)

In [106]:
performance_data = pd.read_csv('loan_performance.csv')

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_51245/225001025.py:1: DtypeWarning: Columns (26,27,28,29,32,35,39,55,56,60,61) have mixed types. Specify dtype option on import or set low_memory=False.
  performance_data = pd.read_csv('loan_performance.csv')


In [107]:
print('Below shows the count of each status')
print(performance_data['ZERO_BALANCE_CODE'].value_counts())
print('---------------------------------')
print('total number of unique loan is', len(performance_data['LOAN_SEQUENCE_NUMBER'].unique()))
print('there is about this many missing zero balance status:',
      len(performance_data['LOAN_SEQUENCE_NUMBER'].unique()) - sum(performance_data['ZERO_BALANCE_CODE'].value_counts()[1:]))

Below shows the count of each status
ZERO_BALANCE_CODE
1.0     72880
9.0       931
16.0      433
3.0       371
96.0      274
2.0       252
15.0      102
Name: count, dtype: int64
---------------------------------
total number of unique loan is 96000
there is about this many missing zero balance status: 20757


/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_51245/3047140243.py:6: FutureWarning: The behavior of obj[i:j] with a float-dtype index is deprecated. In a future version, this will be treated as positional instead of label-based. For label-based slicing, use obj.loc[i:j] instead
  len(performance_data['LOAN_SEQUENCE_NUMBER'].unique()) - sum(performance_data['ZERO_BALANCE_CODE'].value_counts()[1:]))


In [108]:
# If Zero balance code is 1 (prepaid or matured) and the date the balance became zero is before the maturity, the loan is fully prepaid.
condition1 = (performance_data['ZERO_BALANCE_CODE'] ==  1) & (performance_data['ZERO_BALANCE_EFFECTIVE_DATE'] < performance_data['MATURITY_DATE'])
# If zero balance code is 1 and zero balance effective date is the same day as maturity, then loan must be matured. 
condition2 = (performance_data['ZERO_BALANCE_CODE'] == 1) & (performance_data['ZERO_BALANCE_EFFECTIVE_DATE'] == performance_data['MATURITY_DATE'])
# If zero balance code is 3, 2, 96, or 9, it is a default. 
condition3 = performance_data['ZERO_BALANCE_CODE'].isin([9, 96, 3, 2])
# If zero balance code is none of above, we call loan is censored. 
condition4 = performance_data['ZERO_BALANCE_CODE'].isin([15,16])

conditions = [condition1, condition2, condition3, condition4]

values = ['Full Prepayment', 'Matured', 'Default', 'Censored']

# Create the new column using np.select
performance_data['Event_status'] = np.select(conditions, values)
performance_data['Prepayment_status'] = np.select([(performance_data['Event_status']=='Full Prepayment')], [1])
performance_data['Default_status'] = np.select([(performance_data['Event_status'] == 'Default')], [1])

In [109]:
print('Below shows the count of each status')
print(performance_data['Event_status'].value_counts())
print(performance_data['Prepayment_status'].value_counts())
print(performance_data['Default_status'].value_counts())

Below shows the count of each status
Event_status
0                  5163224
Full Prepayment      72705
Default               1828
Censored               535
Matured                158
Name: count, dtype: int64
Prepayment_status
0    5165745
1      72705
Name: count, dtype: int64
Default_status
0    5236622
1       1828
Name: count, dtype: int64


In [110]:
# Create a start and stop time for survival analysis
performance_data['Start_time'] = performance_data['LOAN_AGE']
performance_data['Stop_time'] = performance_data['LOAN_AGE'] + 1

In [111]:
print(min(performance_data['Start_time'].unique()))
print(max(performance_data['Start_time'].unique()))

0
298


In [112]:
# Stratify the loan term
performance_data['ORIGINAL_LOAN_TERM'].value_counts()

ORIGINAL_LOAN_TERM
360    3561290
180    1183419
240     311696
120      89099
300      41655
        ...   
268          8
287          7
246          5
181          4
151          2
Name: count, Length: 154, dtype: int64

In [113]:
condition1 = performance_data['ORIGINAL_LOAN_TERM'].isin(range(0,240))
condition2 = performance_data['ORIGINAL_LOAN_TERM'].isin(range(240,301))
condition3 = performance_data['ORIGINAL_LOAN_TERM'] > 300

conditions = [condition1, condition2, condition3]

values = ['0-19yr', '20-25yr', '+25yr']

performance_data['TERM'] = np.select(conditions, values)

In [114]:
performance_data['TERM'].value_counts()

TERM
+25yr      3581443
0-19yr     1293261
20-25yr     363746
Name: count, dtype: int64

In [115]:
performance_data['CREDIT_SCORE'].replace(9999, np.nan, inplace=True)

# Define the bins and labels for categorical scores
bins = [300, 580, 670, 740, 800, 850]
labels = ['Poor', 'Fair', 'Good', 'Very Good', 'Excellent']

# Create a new column 'credit_score_category' with the categorical scores
performance_data['SCORE'] = pd.cut(performance_data['CREDIT_SCORE'], bins=bins, labels=labels, right=False, include_lowest=True)

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_51245/478400689.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  performance_data['CREDIT_SCORE'].replace(9999, np.nan, inplace=True)


In [116]:
performance_data['SCORE'].value_counts()

SCORE
Very Good    2370025
Good         1676929
Fair          608956
Excellent     544234
Poor           31390
Name: count, dtype: int64

In [117]:
bins = [30000, 60000, 90000, 120000, 150000, 180000, 210000, 240000, 270000, 300000, 500000, 1000000]
labels = ['3-60K', '6-90K', '9-120K', '12-150K', '15-180K', '18-210K', '21-240K', '24-270K', '27-300K', '3-500K', '500k-1M']

# Create a new column 'credit_score_category' with the categorical scores
performance_data['SIZE'] = pd.cut(performance_data['ORIGINAL_UPB'], bins=bins, labels=labels, right=False, include_lowest=True)

In [118]:
performance_data['SIZE'].value_counts()

SIZE
9-120K     761878
12-150K    724263
3-500K     678926
6-90K      664696
15-180K    587449
18-210K    496734
21-240K    378545
3-60K      318147
24-270K    303339
27-300K    229180
500k-1M     72508
Name: count, dtype: int64

In [119]:
performance_data['ORIGINAL_LOAN-TO-VALUE'].replace(999, np.nan, inplace=True)

bins = [20, 40, 60, 80, 100, 120, 150, 200, 500]
labels = ['20-40%', '40-60%', '60-80%', '80-100%', '100-120%', '120-150%', '150-200%', '200-500%']

performance_data['LTV'] = pd.cut(performance_data['ORIGINAL_LOAN-TO-VALUE'], bins=bins, labels=labels, right=False, include_lowest=True)

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_51245/1192586942.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  performance_data['ORIGINAL_LOAN-TO-VALUE'].replace(999, np.nan, inplace=True)


In [120]:
performance_data['LTV'].value_counts()

LTV
80-100%     2045083
60-80%      1808207
40-60%       874597
20-40%       335949
100-120%      90395
120-150%      30752
150-200%      11511
200-500%       3060
Name: count, dtype: int64

In [121]:
print(performance_data['AMORTIZATION_TYPE'].value_counts())
# All of the loans are fixed rate.

AMORTIZATION_TYPE
FRM    5238450
Name: count, dtype: int64


In [122]:
# calculate the average of current interest rate for each month in a given year. 
monthly_avg_interest_rate = performance_data.groupby(['MONTHLY_REPORTING_PERIOD','SCORE'])['CURRENT_INTEREST_RATE'].mean().reset_index()
monthly_avg_interest_rate['CURRENT_INTEREST_RATE'].fillna(performance_data['CURRENT_INTEREST_RATE'].mean(),inplace=True)
monthly_avg_interest_rate.rename(columns={'CURRENT_INTEREST_RATE': 'Avg_Curr_Interest_Rate'}, inplace=True)


performance_data = pd.merge(performance_data, monthly_avg_interest_rate, on = ['MONTHLY_REPORTING_PERIOD', 'SCORE'], how = 'left')
performance_data['ORIGINAL_INTEREST_RATE'].fillna(performance_data['ORIGINAL_INTEREST_RATE'].mean(),inplace=True)

performance_data['Rate_gap'] = performance_data['Avg_Curr_Interest_Rate'] - performance_data['ORIGINAL_INTEREST_RATE']

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_51245/3867774684.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  monthly_avg_interest_rate = performance_data.groupby(['MONTHLY_REPORTING_PERIOD','SCORE'])['CURRENT_INTEREST_RATE'].mean().reset_index()
/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_51245/3867774684.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perfo

In [123]:
print(sum(performance_data['ORIGINAL_INTEREST_RATE'].isna()))
print(sum(performance_data['Rate_gap'].isna()))

0
6916


In [124]:
print(max(performance_data['Rate_gap']))
print(min(performance_data['Rate_gap']))

2.9655021834061133
-5.801557692307693


In [125]:
performance_data['ORIGINAL_LOAN-TO-VALUE'].replace(999, np.nan, inplace=True)

bins = [-6, -2, 0, 1, 2, 3]
labels = ['-6-2%', '-2-0%', '0-1%', '1-2%', '2-3%']

performance_data['Rate_incv'] = pd.cut(performance_data['Rate_gap'], bins=bins, labels=labels, right=False, include_lowest=True)

/var/folders/7j/s81hb8h17wn1zgq5g4yknhzh0000gn/T/ipykernel_51245/3953439698.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  performance_data['ORIGINAL_LOAN-TO-VALUE'].replace(999, np.nan, inplace=True)


In [126]:
performance_data['Rate_incv'].value_counts()

Rate_incv
-2-0%    2358346
0-1%     2259218
1-2%      477533
-6-2%     127476
2-3%        8961
Name: count, dtype: int64

In [127]:
# Make a prepayment rate
# If Zero balance code is 1 (prepaid or matured) and the date the balance became zero is before the maturity, the loan is fully prepaid.
condition1 = (performance_data['ZERO_BALANCE_CODE'] ==  1) & (performance_data['ZERO_BALANCE_EFFECTIVE_DATE'] < performance_data['MATURITY_DATE'])
prepaynum = performance_data[condition1].groupby('MONTHLY_REPORTING_PERIOD')['Prepayment_status'].sum().reset_index(name='Prepayment_Count')

# THis is a condition for default
condition2 = performance_data['ZERO_BALANCE_CODE'].isin([9, 96, 3, 2])
defaultnum = performance_data[condition2].groupby('MONTHLY_REPORTING_PERIOD')['Default_status'].sum().reset_index(name='Default_Count')

# number of total active loans
totalnum = performance_data.groupby('MONTHLY_REPORTING_PERIOD')['LOAN_SEQUENCE_NUMBER'].size().reset_index(name='Loan_count')

# average rate gap, average loan size, average loan term, average credit score, average loan age, average LTV.
averages = performance_data.groupby('MONTHLY_REPORTING_PERIOD').agg({
    'Rate_gap': 'mean',
    'ORIGINAL_UPB': 'mean',
    'ORIGINAL_LOAN_TERM': 'mean',
    'CREDIT_SCORE': 'mean',
    'ORIGINAL_LOAN_TERM': 'mean',
    'ORIGINAL_LOAN-TO-VALUE': 'mean',
    'ORIGINAL_DEBT-TO-INCOME': 'mean',
    'LOAN_AGE': 'mean'
}).reset_index()

# Merge the results
totalnum = pd.merge(totalnum, averages, on='MONTHLY_REPORTING_PERIOD', how='outer')

final_data = pd.merge(totalnum, defaultnum, on='MONTHLY_REPORTING_PERIOD', how='outer')

# Merge the result with prepaynum
final_data = pd.merge(final_data, prepaynum, on='MONTHLY_REPORTING_PERIOD', how='outer')

# Fill missing values with 0
final_data.fillna(0, inplace=True)

final_data['Default_rate'] = 100*final_data['Default_Count'] / final_data['Loan_count']
final_data['Prepay_rate'] = 100*final_data['Prepayment_Count'] / final_data['Loan_count']
final_data

,MONTHLY_REPORTING_PERIOD,Loan_count,Rate_gap,ORIGINAL_UPB,ORIGINAL_LOAN_TERM,CREDIT_SCORE,ORIGINAL_LOAN-TO-VALUE,ORIGINAL_DEBT-TO-INCOME,LOAN_AGE,Default_Count,Prepayment_Count,Default_rate,Prepay_rate
0,199902,107,2.490220e-16,113785.046729,303.476636,717.813084,66.495327,39.289720,0.046729,0.0,0.0,0.000000,0.000000
1,199903,351,-1.796600e-16,117142.450142,292.245014,716.037037,67.917379,35.213675,0.743590,0.0,1.0,0.000000,0.284900
2,199904,711,-8.643767e-17,117125.175809,293.893108,718.344147,69.164557,38.151899,1.156118,0.0,3.0,0.000000,0.421941
3,199905,1099,4.534001e-17,114390.354868,290.283894,716.596171,68.969063,39.635123,1.564149,0.0,2.0,0.000000,0.181984
4,199906,1423,-1.062564e-16,114905.832748,293.352073,714.707952,69.468025,42.669712,2.059733,0.0,4.0,0.000000,0.281096
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,202308,21315,-1.531794e-02,234724.982407,317.494018,747.512835,72.215247,105.880507,64.234107,0.0,153.0,0.000000,0.717804
295,202309,21154,-1.498534e-02,235002.883615,317.641108,747.547522,72.241609,105.514844,65.085752,1.0,101.0,0.004727,0.477451
296,202310,21052,-1.499264e-02,235175.802774,317.742780,747.570655,72.245535,105.358066,66.012683,0.0,114.0,0.000000,0.541516
297,202311,20930,-1.497849e-02,235419.780220,317.826135,747.596349,72.230578,105.086192,66.889489,2.0,77.0,0.009556,0.367893


In [128]:
performance_data.to_csv('LOAN_surv.csv', index=False)
final_data.to_csv('pp_df_rate.csv', index = False)